## *1. CLEANING WORLD HAPPINESS REPORT DATA* ##

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

# adding year column for merge later
whr_2015 = pd.read_csv('../Raw/dataset2_world_happiness_report/2015.csv')
whr_2015['Year'] = 2015

whr_2016 = pd.read_csv('../Raw/dataset2_world_happiness_report/2016.csv')
whr_2016['Year'] = 2016

whr_2017 = pd.read_csv('../Raw/dataset2_world_happiness_report/2017.csv')
whr_2017['Year'] = 2017

whr_2018 = pd.read_csv('../Raw/dataset2_world_happiness_report/2018.csv')
whr_2018['Year'] = 2018

whr_2019 = pd.read_csv('../Raw/dataset2_world_happiness_report/2019.csv')  # Fixed file name
whr_2019['Year'] = 2019

In [ ]:
d1 = pd.read_csv('../Raw/dataset1_avg_annual_hours_worked.csv')

In [ ]:
#renaming columns to common names

whr_2015.rename(columns={
    "Year":"year",
    "Country":"country",
    "Region":"region",
    "Happiness Score": "happiness",
    "Economy (GDP per Capita)": "gdp_per_capita",
    "Family": "social_support",
    "Health (Life Expectancy)": "life_expectancy",
    "Freedom": "freedom",
    "Generosity": "generosity",
    "Trust (Government Corruption)": "govt_corruption"
}, inplace=True)

whr_2016.rename(columns={
    "Year":"year",
    "Country":"country",
    "Region":"region",
    "Happiness Score": "happiness",
    "Economy (GDP per Capita)": "gdp_per_capita",
    "Family": "social_support",
    "Health (Life Expectancy)": "life_expectancy",
    "Freedom": "freedom",
    "Generosity": "generosity",
    "Trust (Government Corruption)": "govt_corruption"
}, inplace=True)

whr_2017.rename(columns={
    "Year":"year",
    "Country":"country",
    "Happiness.Score": "happiness",
    "Economy..GDP.per.Capita.": "gdp_per_capita",
    "Family": "social_support",
    "Health..Life.Expectancy.": "life_expectancy",
    "Freedom": "freedom",
    "Generosity": "generosity",
    "Trust..Government.Corruption.": "govt_corruption"
}, inplace=True)

whr_2018.rename(columns={
    "Year":"year",
    "Country or region":"country",
    "Score": "happiness",
    "GDP per capita": "gdp_per_capita",
    "Social support": "social_support",
    "Healthy life expectancy": "life_expectancy",
    "Freedom to make life choices": "freedom",
    "Generosity": "generosity",
    "Perceptions of corruption": "govt_corruption"
}, inplace=True)

whr_2019.rename(columns={
    "Year":"year",
    "Country or region":"country",
    "Score": "happiness",
    "GDP per capita": "gdp_per_capita",
    "Social support": "social_support",
    "Healthy life expectancy": "life_expectancy",
    "Freedom to make life choices": "freedom",
    "Generosity": "generosity",
    "Perceptions of corruption": "govt_corruption"
}, inplace=True)


In [ ]:
#merging the different dataframes

whr_long = pd.concat([whr_2015, whr_2016, whr_2017, whr_2018, whr_2019], ignore_index=True)

columns_to_keep = ["country", "region", "happiness", "gdp_per_capita", "social_support", "life_expectancy", "freedom", "govt_corruption", "generosity", "year"]

whr_combined = whr_long[columns_to_keep]

In [ ]:
#reordering year col

year_col = whr_combined.pop("year")  # Remove 'Year' column
whr_combined.insert(0, "year", year_col) 

In [ ]:
#Fill missing 'Region' values with region values based on country column in other years
whr_combined["region"] = whr_combined.groupby("country")["region"].ffill().bfill()

In [ ]:
#checking for null values
print(whr_combined.isnull().sum())

In [ ]:
#compute the minimum non-null value of 'govt_corruption'
min_value = whr_combined["govt_corruption"].min()

#fill missing values with the computed minimum
whr_combined["govt_corruption"].fillna(min_value, inplace=True)

In [ ]:
whr_combined

## *2. CLEANING AVG HOURS WORKED PER PERSON PER YEAR* ##

In [ ]:
#changing columns to lowercase
d1.columns = d1.columns.str.lower()

#removing wide spaces
d1.columns = d1.columns.str.replace(' ','_')

#removing unwanted columns
d1.drop(['structure', 'structure_name', 'action', 'ref_area','measure',	
         'measure',	'unit_measure',	'unit_of_measure',
        'obs_status', 'observation_status',	'unit_mult', 'unit_multiplier',	'decimals',	'decimals',
        'sex', 'sex', 'age', 'age',	'labour_force_status',	'labour_force_status',	'work_period',	'work_period', 'aggregation_operation',	
         'aggregation_operation',	'hour_bands', 'hour_bands', 'job_coverage','job_coverage',
        'hours_type', 'hours_type',	'worker_status', 'worker_status', 'work_time_arngmnt',	'working_time_arrangement',
        'structure_id','observation_value'], axis=1, inplace=True)

#removing the second value time_period column, since its  duplicated
d1 = d1.iloc[:, ~d1.columns.duplicated()]

# Reset the index, turn it into a column
d1 = d1.reset_index()
whr_combined = whr_combined.reset_index()

whr_combined = whr_combined.reset_index()
# Rename the index column properly
d1 = d1.rename(columns={'index': 'hours_id', 'reference_area': 'country', 'obs_value': 'average_hours', 'time_period': 'year'})
whr_combined = whr_combined.rename(columns ={'index': 'whr_id'})


In [ ]:
display(d1)
display(whr_combined)

## *3. CREATE COUNTRIES & REGIONS TABLE* ##

In [ ]:
#replacing inconsistent names
d1['country'].replace("Türkiye", "Turkey", inplace=True)
d1['country'].replace("Slovak Republic", "Slovakia", inplace=True)
whr_combined['country'].replace("Czech Republic", "Czechia", inplace=True)


In [ ]:
#dropping countries not present on both tables

whr_combined['country'].replace([
    'Somalia', 'Somaliland Region', 'Somaliland Region',
    'Taiwan', 'Taiwan Province of China', 'Trinidad & Tobago',
    'Trinidad and Tobago', 'Palestinian Territories',
    'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Hong Kong S.A.R., China','Hong Kong'
], np.nan, inplace=True)

#drop NaN values
whr_combined = whr_combined.dropna(subset=['country'])

In [ ]:
#dropping countries not present on both tables

d1['country'].replace(['OECD'], np.nan, inplace=True)

d1 = d1.dropna(subset=['country'])

In [ ]:
#Creating countries dataframe

countries = whr_combined[['country']].drop_duplicates().reset_index(drop=True)

countries['country_id'] = countries.index

In [ ]:
countries

In [ ]:
#Creating regions dataframe

regions = whr_combined[['region']].drop_duplicates().reset_index(drop=True)

regions['region_id'] = regions.index

In [ ]:
# Merge the two dataframes to replace country/region names with country_id and region_id
whr_combined = whr_combined.merge(regions, on="region", how="left")
whr_combined = whr_combined.merge(countries, on="country", how="left")


# Drop the original 'country' column and rename 'country_id' appropriately
whr_combined.drop(columns=['country','level_0','region'], inplace=True)


In [ ]:
#reorder columns
cols = whr_combined.columns.tolist()
cols.insert(1, cols.pop(cols.index('country_id')))
cols.insert(2, cols.pop(cols.index('region_id')))
whr_combined = whr_combined[cols]

display(whr_combined)

In [ ]:
# Merge the two dataframes to replace country/region names with country_id and region_id
d1 = d1.merge(countries, on="country", how="left")

d1.drop(columns=['country'], inplace=True)

In [ ]:
cols = d1.columns.tolist()
cols.insert(1, cols.pop(cols.index('country_id')))
d1 = d1[cols]

In [ ]:
#country_id column is a float because it contains NaN values due to them being countries not present in countries.df

d1 = d1.dropna(subset=['country_id'])  # Remove NaN values
d1['country_id'] = d1['country_id'].astype(int)  # Convert to integer

In [ ]:
# set index=false when exporting dataframe .replace("\n", " ")
#d1['country'] = d1['country'].apply(lambda s: s.replace("\n", " "))

#convert regions to csv
regions['region'] = regions['region'].apply(lambda s: s.replace("\n"," "))
regions.to_csv('regions.csv', index=False, sep=";", encoding="utf-8")

In [ ]:
#convert countries to csv
countries['country'] = countries['country'].apply(lambda s: s.replace("\n"," "))
countries.to_csv('countries.csv', index=False, sep=";", encoding="utf-8")

In [ ]:
#convert whr to csv
whr_combined.to_csv('whr_combined.csv', index=False, sep=";", encoding="utf-8")

In [ ]:
#convert avg hours to csv
d1.to_csv('average_hours_worked.csv', index=False, sep=";", encoding="utf-8")